In [38]:
import pandas as pd

df = pd.read_json("subclass-sample.json")

print(df['result'])

import json
# load data using Python JSON module
with open('subclass-sample.json','r') as f:
    data = json.loads(f.read())

df_nested_list = pd.json_normalize(data, record_path =['result'])



0     {'_id': '158.csv', 'subclasses': ['FOSFOMYCIN'...
1     {'_id': '124.csv', 'subclasses': ['TETRACYCLIN...
2     {'_id': '120.csv', 'subclasses': ['FOSFOMYCIN'...
3     {'_id': '186.csv', 'subclasses': ['TETRACYCLIN...
4     {'_id': '187.csv', 'subclasses': ['BETA-LACTAM...
                            ...                        
93    {'_id': '109.csv', 'subclasses': ['TETRACYCLIN...
94    {'_id': '133.csv', 'subclasses': ['FOSFOMYCIN'...
95    {'_id': '138.csv', 'subclasses': ['FOSFOMYCIN'...
96    {'_id': '142.csv', 'subclasses': ['FOSFOMYCIN'...
97    {'_id': '152.csv', 'subclasses': ['BETA-LACTAM...
Name: result, Length: 98, dtype: object


In [39]:
print(df_nested_list.head(5))

# group by account, then list all services
transactions = df_nested_list.groupby(['_id'])['subclasses'].apply(list)

print(transactions.describe())

from apyori import apriori

transactions = df_nested_list
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.05))


# print first 5 rules
print(results[:5])


       _id                                         subclasses
0  158.csv  [FOSFOMYCIN, FOSFOMYCIN, TETRACYCLINE, BETA-LA...
1  124.csv  [TETRACYCLINE, BETA-LACTAM, BETA-LACTAM, BETA-...
2  120.csv  [FOSFOMYCIN, BETA-LACTAM, BETA-LACTAM, BETA-LA...
3  186.csv  [TETRACYCLINE, FOSFOMYCIN, QUINOLONE, BLEOMYCI...
4  187.csv  [BETA-LACTAM, BETA-LACTAM, QUINOLONE, METHICIL...
count                                                    98
unique                                                   40
top       [[FOSFOMYCIN, BETA-LACTAM, BETA-LACTAM, BETA-L...
freq                                                     18
Name: subclasses, dtype: object
[RelationRecord(items=frozenset({'_'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'_'}), confidence=0.5, lift=1.0)]), RelationRecord(items=frozenset({'a'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'a'}), confidence=0.5, lift=1.0)]), RelationRecord(

In [40]:
transactions = pd.DataFrame(df_nested_list)

transactions = df_nested_list.groupby(['_id'])['subclasses'].apply(list)

transaction_list = df_nested_list['subclasses'].apply(list)
transaction_list = list(transaction_list)

results = list(apriori(transaction_list, min_support=0.05))

# print first 5 rules
print(results[:5])







[RelationRecord(items=frozenset({'BETA-LACTAM'}), support=0.9693877551020408, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'BETA-LACTAM'}), confidence=0.9693877551020408, lift=1.0)]), RelationRecord(items=frozenset({'FOSFOMYCIN'}), support=0.8061224489795918, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'FOSFOMYCIN'}), confidence=0.8061224489795918, lift=1.0)]), RelationRecord(items=frozenset({'MACROLIDE'}), support=0.15306122448979592, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'MACROLIDE'}), confidence=0.15306122448979592, lift=1.0)]), RelationRecord(items=frozenset({'METHICILLIN'}), support=0.2857142857142857, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'METHICILLIN'}), confidence=0.2857142857142857, lift=1.0)]), RelationRecord(items=frozenset({'QUINOLONE'}), support=0.16326530612244897, ordered_statistics=[OrderedStatistic(items_base

In [41]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(result_df.head(20))

      Left_side                Right_side   Support  Confidence      Lift
0                             BETA-LACTAM  0.969388    0.969388  1.000000
1                              FOSFOMYCIN  0.806122    0.806122  1.000000
2                               MACROLIDE  0.153061    0.153061  1.000000
3                             METHICILLIN  0.285714    0.285714  1.000000
4                               QUINOLONE  0.163265    0.163265  1.000000
5                            TETRACYCLINE  1.000000    1.000000  1.000000
6                  FOSFOMYCIN,BETA-LACTAM  0.775510    0.775510  1.000000
7   BETA-LACTAM                FOSFOMYCIN  0.775510    0.800000  0.992405
8    FOSFOMYCIN               BETA-LACTAM  0.775510    0.962025  0.992405
9                   BETA-LACTAM,MACROLIDE  0.153061    0.153061  1.000000
10  BETA-LACTAM                 MACROLIDE  0.153061    0.157895  1.031579
11    MACROLIDE               BETA-LACTAM  0.153061    1.000000  1.031579
12                METHICILLIN,BETA-LAC

In [45]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(20))

result_df.to_csv('association_rule.csv', index=False)

                                          Left_side  \
155                                       QUINOLONE   
221                          TETRACYCLINE,QUINOLONE   
247              TETRACYCLINE,QUINOLONE,BETA-LACTAM   
111                                       QUINOLONE   
158                           QUINOLONE,BETA-LACTAM   
241                          TETRACYCLINE,QUINOLONE   
230                                       QUINOLONE   
234                           QUINOLONE,BETA-LACTAM   
214                                       QUINOLONE   
249              TETRACYCLINE,METHICILLIN,MACROLIDE   
216                           METHICILLIN,MACROLIDE   
162               METHICILLIN,BETA-LACTAM,MACROLIDE   
253  TETRACYCLINE,METHICILLIN,BETA-LACTAM,MACROLIDE   
159                           METHICILLIN,MACROLIDE   
236                           METHICILLIN,MACROLIDE   
112                           METHICILLIN,MACROLIDE   
242               METHICILLIN,BETA-LACTAM,MACROLIDE   
223       